In [8]:
# SAVOIR UTILISER GPU SALLE TP (407 509 502)
# entrainer modele generatif avec requete reformulée et requete non reformulée avec splade avec jeu de données de reformulation de requetes (CANARD)
# voir encodage et voir re-ranking (pour les améliorer)

In [10]:
import ir_datasets
import ir_measures
import logging
import os
import multiprocessing

import pyterrier as pt
# pt.init()

import pytrec_eval
from experimaestro import experiment
from datamaestro import prepare_dataset
from xpmir.measures import Measure, AP, nDCG, RR, P
from xpmir.interfaces.anserini import IndexCollection, AnseriniRetriever
from xpmir.rankers.standard import BM25
from xpmir.evaluation import Evaluate
from transformers import GPT2Tokenizer, GPT2Model, TFGPT2Model, AutoTokenizer, AutoModelForSeq2SeqLM
from xpmir.models import AutoModel
from xpmir.neural.colbert import Colbert
from xpmir.neural.splade import spladeV2_doc, spladeV2_max

import torch
import numpy as np

In [ ]:
# GPT2 PYTORCH MODEL

tokenizer_gpt2 = GPT2Tokenizer.from_pretrained('gpt2')
model_gpt2pt = GPT2Model.from_pretrained('gpt2')

In [4]:
# GPT2 TENSORFLOW MODEL

tokenizer_gpt2 = GPT2Tokenizer.from_pretrained('gpt2')
model_gpt2tf = TFGPT2Model.from_pretrained('gpt2')

All model checkpoint layers were used when initializing TFGPT2Model.

All the layers of TFGPT2Model were initialized from the model checkpoint at gpt2.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2Model for predictions without further training.


In [5]:
# T5 HUGGINGFACE MODEL

tokenizer_t5 = AutoTokenizer.from_pretrained("t5-base")

model_t5 = AutoModelForSeq2SeqLM.from_pretrained("t5-base")

/home/gameselo/miniconda3/envs/mapsi/lib/python3.9/site-packages/transformers/models/t5/tokenization_t5_fast.py:155: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


In [12]:
# MONOT5 RE-RANKER MODEL (MSMARCO FINE-TUNED)

tokenizer_monot5 = AutoTokenizer.from_pretrained("castorini/monot5-base-msmarco-10k")

model_monot5 = AutoModelForSeq2SeqLM.from_pretrained("castorini/monot5-base-msmarco-10k")

In [11]:
# MONOBERT

model_monobert = AutoModel.load_from_hf_hub("xpmir/monobert")

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

In [16]:
logging.basicConfig(level=logging.INFO)
dataset_xpmir = prepare_dataset("irds.vaswani")

dataset2020 = prepare_dataset("irds.trec-cast.v1.2020")

In [16]:
# DATASETS FOR TREC 2021
dataset2 = prepare_dataset("irds.msmarco-document.trec-dl-2020")
dataset3 = ir_datasets.load("wapo/v3/trec-news-2020")
dataset4 = prepare_dataset("irds.kilt.codec")

In [19]:
# EXPERIMENTS

logging.getLogger().setLevel(logging.INFO)

with experiment("experiments", "bm25", port=12345) as xp:
    xp.setenv("JAVA_HOME", os.environ["JAVA_HOME"])
    
    index = IndexCollection(documents=dataset2020.documents, storePositions=True, storeDocvectors=True, storeContents=True, threads=6).submit()
    bm25_retriever = AnseriniRetriever(k=1000, index=index, model=BM25())
    bm25_eval = Evaluate(dataset=dataset2020, retriever=bm25_retriever, measures=[
        AP@10, nDCG@3, RR, nDCG@10, R@10
    ]).submit()

print("BM25 results")
print(bm25_eval.results.read_text())

INFO:xpm:Submitting job Job[a23d25d9fa3cbc0f1047ec4440c40ae0e2eddca3c5b3115ae2807ce359843090]
INFO:xpm:Starting job /home/gameselo/Bureau/M1_DAC_2022_2023/PLDAC/experiments/jobs/xpmir.interfaces.anserini.indexcollection/a23d25d9fa3cbc0f1047ec4440c40ae0e2eddca3c5b3115ae2807ce359843090
INFO:xpm:Process started (Process(13920))
INFO:xpm:Submitting job Job[4a6e21090ad3fd5ff63c29a7898bc7b7f98b9b17647374b6f654f0235a305d05]


BM25 results


FileNotFoundError: [Errno 2] No such file or directory: '/home/gameselo/Bureau/M1_DAC_2022_2023/PLDAC/experiments/jobs/xpmir.evaluation.evaluate/4a6e21090ad3fd5ff63c29a7898bc7b7f98b9b17647374b6f654f0235a305d05/aggregated.txt'